In [1]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

PS=32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [2]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 100
from torch.utils.data import random_split, DataLoader

train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.3,0.5,0.2])
# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=True)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=True)

In [3]:
descriptor = kornia.feature.SIFTDescriptor(PS, rootsift=True).to(device).eval()

with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, descriptor,th=0.8)
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Positivos: {total_acertos} Negativos: {total_erros}')


100%|██████████| 13/13 [00:00<00:00, 31.29it/s]

Total de elementos no DataLoader: 4944
Positivos: 1239 Negativos: 16


In [4]:
from torch import nn
from external.hardnet_pytorch import HardNet 

class Descriptor(nn.Module):
        def __init__(self) -> None:
           super().__init__()           
           hardnet = HardNet()
           checkpoint = torch.load('trained_models/pretrained_nets/HardNet++.pth')
           hardnet.load_state_dict(checkpoint['state_dict'])
           self.model = hardnet
        def forward(self, x):
           return self.model(x)
    
descriptor = Descriptor().to(device).eval()


total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, descriptor,th=0.8)
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Positivos: {total_acertos} Negativos: {total_erros}')

100%|██████████| 13/13 [00:00<00:00, 96.90it/s]

Total de elementos no DataLoader: 4944
Positivos: 1436 Negativos: 12
